In [10]:
    import svgwrite

In [136]:
    import flexx.pyscript
    import traitlets, IPython, mistune
    import inspect
    import textwrap
    import collections
    import inspect, abc, sys

    class Maus(abc.ABCMeta):        
        def __call__(Maus, object=None):
            if isinstance(object, str):
                object = mistune.markdown(object)
                start, sep, end = object.partition('>')
                return IPython.display.HTML(F"{start} {str(Maus)}{sep}{end}")
            if 'vdom' in sys.modules and isinstance(object, __import__('vdom').VDOM):
                kwargs = object.to_dict()
                kwargs['attributes'] = kwargs.get('attributes', {})
                kwargs['attributes'].update(Maus.to_dict())
                tagName = kwargs.pop('tagName')
                return vdom.VDOM(tagName, **kwargs)
            if 'svgwrite' in sys.modules and isinstance(object, svgwrite.base.BaseElement):
                object.update(Maus.to_dict(True))
                return object
            if 'bs4' in sys.modules and isinstance(object, __import__('bs4').element.Tag):
                object.attrs.update(Maus.to_dict(True))
                return object
        
        def dispatch(Maus, object):
            if callable(object): object = flexx.pyscript.py2js(textwrap.dedent('\n'.join(inspect.getsource(object).splitlines()[1:])))
            return object

        def to_dict(self, dispatch=False, **data):
            for key in (x for x in dir(self) if x not in dir(MausInstance)):
                data[key] = getattr(self, key)
                if dispatch: data[key] = self.dispatch(data[key])
            data.update({F'data-{key}': value for key, value in getattr(self, 'data', {}).items()})
            return data
            
        def __str__(self):
            return " ".join(F'{key}="{value}"' for key, value in self.to_dict(True).items())
    maus = {'metaclass': Maus}
    class MausInstance(**maus): ...